<a href="https://colab.research.google.com/github/jakob-steuer/chemoinformatics_2025/blob/main/chemoinformatics_4_chemical_databases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created with Google Gemini v2.5 Pro

In [ ]:
pip install rdkit requests

# Databases

# Traveling Salesman Algoritm

This section is based on this resource: https://www.w3schools.com/dsa/dsa_ref_traveling_salesman.php

In [ ]:
from itertools import permutations

## Brute Force Algorithm

To find the optimal solution to The Traveling Salesman Problem, we will check all possible routes, and every time we find a shorter route, we will store it, so that in the end we will have the shortest route.

Good: Finds the overall shortest route.

Bad: Requires an awful lot of calculation, especially for a large amount of cities, which means it is very time consuming.

In [ ]:
def calculate_distance(route, distances):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distances[route[i]][route[i + 1]]
    total_distance += distances[route[-1]][route[0]]
    return total_distance

def brute_force_tsp(distances):
    n = len(distances)
    cities = list(range(1, n))
    shortest_route = None
    min_distance = float('inf')

    for perm in permutations(cities):
        current_route = [0] + list(perm)
        current_distance = calculate_distance(current_route, distances)

        if current_distance < min_distance:
            min_distance = current_distance
            shortest_route = current_route

    shortest_route.append(0)
    return shortest_route, min_distance

distances = [
    [0, 2, 2, 5, 9, 3],
    [2, 0, 4, 6, 7, 8],
    [2, 4, 0, 8, 6, 3],
    [5, 6, 8, 0, 4, 9],
    [9, 7, 6, 4, 0, 10],
    [3, 8, 3, 9, 10, 0]
]

distances

In [ ]:
route, total_distance = brute_force_tsp(distances)
print("Route:", route)
print("Total distance:", total_distance)

## Greedy Algorithm

Using A Greedy Algorithm to Solve The Traveling Salesman Problem
Since checking every possible route to solve the Traveling Salesman Problem (like we did above) is so incredibly time consuming, we can instead find a short route by just going to the nearest unvisited city in each step, which is much faster.

Good: Finds a solution to the Traveling Salesman Problem much faster than by checking all routes.

Bad: Does not find the overall shortest route, it just finds a route that is much shorter than an average random route.

In [ ]:
def nearest_neighbor_tsp(distances):
    n = len(distances)
    visited = [False] * n
    route = [0]
    visited[0] = True
    total_distance = 0

    for _ in range(1, n):
        last = route[-1]
        nearest = None
        min_dist = float('inf')
        for i in range(n):
            if not visited[i] and distances[last][i] < min_dist:
                min_dist = distances[last][i]
                nearest = i
        route.append(nearest)
        visited[nearest] = True
        total_distance += min_dist

    total_distance += distances[route[-1]][0]
    route.append(0)
    return route, total_distance

distances = [
    [0, 2, 2, 5, 9, 3],
    [2, 0, 4, 6, 7, 8],
    [2, 4, 0, 8, 6, 3],
    [5, 6, 8, 0, 4, 9],
    [9, 7, 6, 4, 0, 10],
    [3, 8, 3, 9, 10, 0]
]

In [ ]:
route, total_distance = nearest_neighbor_tsp(distances)
print("Route:", route)
print("Total distance:", total_distance)

## Actual Traveling Salesman Problems Are More Complex

The edge weight in a graph in this context of The Traveling Salesman Problem tells us how hard it is to go from one point to another, and it is the total edge weight of a route we want to minimize.

So far on this page, the edge weight has been the distance in a straight line between two points. And that makes it much easier to explain the Traveling Salesman Problem, and to display it.

But in the real world there are many other things that affects the edge weight:

- Obstacles: When moving from one place to another, we normally try to avoid obstacles like trees, rivers, houses for example. This means it is longer and takes more time to go from A to B, and the edge weight value needs to be increased to factor that in, because it is not a straight line anymore.
- Transportation Networks: We usually follow a road or use public transport systems when traveling, and that also affects how hard it is to go (or send a package) from one place to another.
- Traffic Conditions: Travel congestion also affects the travel time, so that should also be reflected in the edge weight value.
- Legal and Political Boundaries: Crossing border for example, might make one route harder to choose than another, which means the shortest straight line route might be slower, or more costly.
- Economic Factors: Using fuel, using the time of employees, maintaining vehicles, all these things cost money and should also be factored into the edge weights.

# Exploring Chemical Databases with APIs

In our lecture, we discussed how chemical databases like PubChem store information on millions of molecules. But how do we access this data from a script? We could download massive files, but a more efficient way is to use an Application Programming Interface, or API.

## API

The Restaurant Analogy

    Think of an API as a waiter in a restaurant.

    You (the client) want to get data from a web service (the kitchen).

    You don't go into the kitchen yourself. Instead, you give your order to the waiter (the API) using a structured format (the menu).

    The waiter takes your request to the kitchen, which prepares your order.

    The waiter then brings the data (the food) back to you.

For web-based databases, these requests are often made by constructing a special URL. This is known as a REST API.

## REST API

A REST API is a standardized way for computer systems to communicate with each other over the internet. REST stands for Representational State Transfer, and an API is an Application Programming Interface.

**How It Works: URLs as Requests**

A REST API works by constructing a specific URL to ask for data. This URL contains all the information the server needs to fulfill the request.

For example, to get the SMILES string for ethanol from PubChem, you would construct a URL like this:
https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/CCO/cids/txt

This URL is composed of several parts:

    Endpoint: https://pubchem.ncbi.nlm.nih.gov/rest/pug/ - The base address of the API.

    Path: /compound/smiles/CCO/cids - Specifies exactly what you want (the CID for the compound with the SMILES string "CCO").

    Output Format: /txt - Tells the server to return the data as plain text.

When you send this request (e.g., in a web browser or a Python script), the PubChem server sends back the requested data. This structured, URL-based approach is what makes REST APIs so powerful and easy to use for programmatic data access.

Today, we'll use the PubChem PUG-REST API to fetch data directly into our notebook.

## PUG-REST API
A more detailed introduction to the PUG-Rest API can be found here:

https://pubchem.ncbi.nlm.nih.gov//docs/pug-rest-tutorial#section=How-PUG-REST-Works

In [ ]:
# Import necessary libraries
import requests
from rdkit import Chem
from rdkit.Chem import Draw
from IPython.display import display
import time
import json

## Identity Search: Fetching a Molecule by Name

The simplest query is a text search. Let's ask PubChem for the structure of "Aspirin" and some of its properties.

The API request is built into the URL. We'll ask for the molecule's SMILES string and its molecular weight.

In [ ]:
# The base URL for PubChem's API
base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"

# Let's search for Aspirin by its name
compound_name = "Aspirin"

# First, get the SMILES string
smiles_url = f"{base_url}/compound/name/{compound_name}/property/CanonicalSMILES/txt"
smiles_response = requests.get(smiles_url)
aspirin_smiles = smiles_response.text.strip()

# Second, get the Molecular Weight
mw_url = f"{base_url}/compound/name/{compound_name}/property/MolecularWeight/txt"
mw_response = requests.get(mw_url)
aspirin_mw = mw_response.text.strip()

In [ ]:
print(f"--- Processed Data ---")
print(f"Name: {compound_name}")
print(f"SMILES: {aspirin_smiles}")
print(f"Molecular Weight: {aspirin_mw} g/mol")

# Now, let's visualize the structure using RDKit
aspirin_mol = Chem.MolFromSmiles(aspirin_smiles)
display(aspirin_mol)

### Exercise 1:

Investigate what happens when you copy-paste the SMILES or Molecular Weight  URLs in a Browser

### Exercise 2:

Extract the InChI, the InChIKey and an additional property of your choice (see the `Compound Property Tables` here: https://pubchem.ncbi.nlm.nih.gov/docs/pug-rest) from the PubChem API.

## Substructure Search: Finding Molecules with a Specific Fragment

A more powerful query is a substructure search (SSS).

Let's find molecules in PubChem that contain the **coumarin** scaffold. It's a bicyclic structure that's the basis for many natural products and pharmaceuticals (like the anticoagulant Warfarin).

This is a two-step process:

- Send a request with the substructure (as a SMILES string). PubChem will start the search and give us a "ListKey" that acts as a ticket to our results.

- Use that ListKey to retrieve the actual compound IDs (CIDs) of the molecules that matched.

In [ ]:
# Step 1: Initiate the substructure search to get a ListKey

substructure_smiles = "C1=CC=C2C(=C1)C=CC(=O)O2"
base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
search_url = f"{base_url}/compound/substructure/smiles/{substructure_smiles}/JSON"

print("Sending search request to PubChem...")

try:
    # Initiate the search
    response = requests.post(search_url)
    response.raise_for_status()
    initial_data = response.json()

    if 'Waiting' not in initial_data:
        print("Error: Could not initiate search.")
        print("Server response:", initial_data)
    else:
        listkey = initial_data['Waiting']['ListKey']
        print(f"Search successfully initiated. Received ListKey: {listkey}")

        # Step 2: Poll the server until the results are ready
        retrieve_url = f"{base_url}/compound/listkey/{listkey}/cids/JSON"
        print("Polling server for results... (this may take a moment)")

        status = "Waiting"
        while status == "Waiting":
            # Wait for a couple of seconds between checks
            time.sleep(2)

            results_response = requests.get(retrieve_url)
            results_response.raise_for_status()
            results_data = results_response.json()

            # Check if the 'Waiting' key is still present in the response
            if 'Waiting' in results_data:
                print("...search is still running.")
            else:
                status = "Ready" # Exit the loop

        # Once the loop exits, the results are ready to be processed
        print("\n--- Search Complete! ---")

        if 'IdentifierList' in results_data:
            compound_ids = results_data['IdentifierList']['CID']
            print(f"Found {len(compound_ids)} molecules containing the catechol fragment.")
            print(f"Here are the first 10 Compound IDs (CIDs):")
            print(compound_ids[:10])
        else:
            print("Search finished, but no identifiers were found.")
            print("Server response:", results_data)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
except KeyError as e:
    print(f"A key was not found in the response: {e}")

## Similarity Search: Finding Structurally Similar Molecules

Finally, let's perform a similarity search. We will find molecules that are structurally similar to Dopamine. This search relies on comparing molecular fingerprints.

The process is very similar to a substructure search.

In [ ]:
from rdkit.Chem import Draw

# Define our base URL and query molecule
base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
dopamine_smiles = "C1=CC(=C(C=C1CCN)O)O"
dopamine_cid = 681 # Known CID for Dopamine

print("--- Step 1: Initiating Asynchronous Similarity Search ---")

# --- CORRECTED PART ---
# The SMILES and Threshold must be part of the URL itself for the POST request.
search_url = f"{base_url}/compound/similarity/smiles/{dopamine_smiles}/JSON?Threshold=95"
print(f"Using search URL: {search_url}")

try:
    # Initiate the search via POST to the constructed URL. The POST body is empty.
    response = requests.post(search_url)
    response.raise_for_status()
    initial_data = response.json()

    if 'Waiting' not in initial_data:
        print("Error: Could not initiate search.")
        print("Server response:", initial_data)
        listkey = None
    else:
        listkey = initial_data['Waiting']['ListKey']
        print(f"Search successfully initiated. Received ListKey: {listkey}")

    if listkey:
        # --- Step 2: Poll Server for Results ---
        retrieve_url = f"{base_url}/compound/listkey/{listkey}/cids/JSON"
        print("Polling server for results... (this may take a moment)")

        status = "Waiting"
        while status == "Waiting":
            time.sleep(2) # Wait between checks

            results_response = requests.get(retrieve_url)
            results_response.raise_for_status()
            results_data = results_response.json()

            if 'Waiting' in results_data:
                print("...search is still running.")
            else:
                status = "Ready"

        print("\n--- Search Complete! ---")

        # --- Step 3: Retrieve and Visualize Top Hits ---
        if 'IdentifierList' in results_data:
            # Get the top 5 similar compound IDs
            similar_cids = results_data['IdentifierList']['CID'][:5]
            print(f"Found top 5 similar CIDs: {similar_cids}")

            # Prepare CIDs for property fetching (Query + Hits)
            all_cids_to_draw = [str(dopamine_cid)] + [str(cid) for cid in similar_cids]
            cids_str = ",".join(all_cids_to_draw)

            # Fetch SMILES for all molecules in a single request
            properties_url = f"{base_url}/compound/cid/{cids_str}/property/CanonicalSMILES/JSON"
            prop_res = requests.get(properties_url)
            prop_data = prop_res.json()['PropertyTable']['Properties']

            # Create a dictionary of {CID: SMILES}
            cid_smiles_map = {item['CID']: item['CanonicalSMILES'] for item in prop_data}

            # Prepare data for RDKit, ensuring Dopamine is first
            mols_to_draw = [Chem.MolFromSmiles(cid_smiles_map[cid]) for cid in [dopamine_cid] + similar_cids]
            legends = [f"Dopamine (Query)"] + [f"CID: {cid}" for cid in similar_cids]

            # Generate and save the image
            img = Draw.MolsToGridImage(mols_to_draw, molsPerRow=3, subImgSize=(300, 300), legends=legends)
            img.save('similarity_hits.png')
            print("\nImage 'similarity_hits.png' has been generated successfully.")

        else:
            print("Search finished, but no identifiers were found.")

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
except KeyError as e:
    print(f"A key was not found in the response: {e}")

## Exercise 1

Modify the code in the last section to perform a similarity search for Paracetamol (also known as Acetaminophen).

Find the SMILES string for Paracetamol.

Use it to perform a similarity search with a Tanimoto threshold of 90 (Threshold=90).

Retrieve and visualize the top 5 most similar molecules from PubChem.